In [161]:
#dataframes
import pandas as pd
import h5py
import datetime as dt

#math
import numpy as np
import math
import scipy.fftpack
from scipy import signal
import scipy.interpolate as interpolate
from scipy import integrate
pd.TimeSeries = pd.Series 

#gps
from geopy import distance

#plots
import pylab as plt
import matplotlib.pyplot as plt
import matplotlib.animation as animation

### Stationary Wind Bag B

In [230]:
#mac 
fname = '~/Documents/Myfiles/DataAnalysis/data/20201020_172919_windy_b_data.hdf'
wind=pd.read_hdf(fname)
wind.keys()

Index(['millis', 'lat', 'lon', 'gps_time', 'gps_date', 'wind', 'gps_date_str',
       'year', 'month', 'day', 'gps_time_str', 'hour', 'minute', 'second',
       'time_epoch', 'S2', 'D', 'U', 'V', 'W', 'T', 'H', 'AD', 'MD',
       'corrected_u', 'corrected_v'],
      dtype='object')

In [224]:
pi = np.pi/180
x = np.abs(wind.S2*np.sin(((wind.D+13)*pi)))
y = np.abs(wind.S2*np.cos(((wind.D+13)*pi)))

In [225]:
x = x*np.sign(wind.U)
y = y*np.sign(wind.V)

In [226]:
s = []
for i in range(len(x)):
    if(np.sign(x[i])!=np.sign(wind.U[i])):
        s.append(i)


In [227]:
wind['corrected_u'] = x
wind['corrected_v'] = y

In [229]:
wind.to_hdf(fname, key='wind',mode='w')

/Users/an/Documents/MyFiles/DataAnalysis/FlyDataAnalysis/lib/python3.8/site-packages/pandas/core/generic.py:2606: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->Index(['wind', 'gps_date_str', 'gps_time_str'], dtype='object')]

  pytables.to_hdf(


### Extract Wind Data for shorter duration

In [219]:
df =pd.DataFrame
df = wind.loc[(wind.time_epoch >= sensor.master_time.iloc[0]) & (wind_df.time_epoch <=sensor.master_time.iloc[-1])]

In [217]:
sensor= pd.read_hdf('~/Documents/Myfiles/DataAnalysis/data/Sprints/Run03/Collections/Run03Coll01_InterpolatedX.hdf')